In [1]:
import cv2
import os

def create_video_from_images_cv2(image_folder, output_video_path, fps):
    """
    Creates an MP4 video from a sequence of images using OpenCV.

    Args:
        image_folder (str): Path to the folder containing image frames.
        output_video_path (str): Path and filename for the output MP4 video.
        fps (int): Frames per second for the output video.
    """
    images = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
    images.sort() # Ensure images are sorted correctly if they have sequential names (e.g., frame_001.png, frame_002.png)

    if not images:
        print(f"No images found in {image_folder}")
        return

    # Read the first image to get dimensions
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    # For MP4, 'mp4v' or 'XVID' often works well. 'avc1' is another common one.
    # Note: Codecs can sometimes be platform-dependent.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can try 'XVID', 'MJPG', 'DIVX'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    if not out.isOpened():
        print(f"Error: Could not open video writer for {output_video_path}. Check codec availability.")
        return

    for image_name in images:
        image_path = os.path.join(image_folder, image_name)
        frame = cv2.imread(image_path)
        if frame is not None:
            out.write(frame)
        else:
            print(f"Warning: Could not read image {image_path}. Skipping.")

    out.release()
    print(f"Video '{output_video_path}' created successfully from {len(images)} images.")

if __name__ == "__main__":
    # Example Usage:
    # 1. Create a dummy folder with some image files for testing
    dummy_image_folder = "frames"
    os.makedirs(dummy_image_folder, exist_ok=True)

    # Create some dummy image files (e.g., using PIL for demonstration)
    try:
        from PIL import Image, ImageDraw, ImageFont
        for i in range(30): # Create 30 frames
            img = Image.new('RGB', (640, 480), color = (73, 109, 137))
            d = ImageDraw.Draw(img)
            d.text((20,20), f"Frame {i+1}", fill=(255,255,0))
            img.save(os.path.join(dummy_image_folder, f"frame_{i:03d}.png"))
        print(f"Created dummy images in '{dummy_image_folder}'")
    except ImportError:
        print("Pillow not installed. Please manually create some .png or .jpg images in 'temp_frames' for testing.")
        print("Example: frame_001.png, frame_002.png, ...")
        # Exit if PIL is not available to avoid error for dummy image creation
        exit()


    output_video_file = "output_video_cv2.mp4"
    frame_rate = 10 # frames per second

    create_video_from_images_cv2(dummy_image_folder, output_video_file, frame_rate)

    # Clean up dummy folder (optional)
    # import shutil
    # shutil.rmtree(dummy_image_folder)
    # print(f"Cleaned up dummy folder: {dummy_image_folder}")


Created dummy images in 'frames'


AttributeError: module 'cv2' has no attribute 'VideoWriter_fourcc'

In [2]:
from PIL import Image
import os

def create_gif_from_pngs_pillow(image_folder, output_gif_path, duration=100, loop=0):
    """
    Creates a GIF from a sequence of PNG images using Pillow.

    Args:
        image_folder (str): Path to the folder containing PNG image frames.
        output_gif_path (str): Path and filename for the output GIF.
        duration (int): Duration of each frame in milliseconds (e.g., 100ms = 0.1 seconds).
        loop (int): Number of times the GIF should loop. 0 means infinite loop.
    """
    png_files = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(".png")]
    png_files.sort() # Ensure images are sorted correctly (e.g., frame_001.png, frame_002.png)

    if not png_files:
        print(f"No PNG images found in {image_folder}")
        return

    # Open the first image
    try:
        frames = []
        for file_path in png_files:
            try:
                img = Image.open(file_path)
                # Ensure all images are converted to RGB if they are not already (GIFs typically work better with RGB/P)
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                frames.append(img)
            except Exception as e:
                print(f"Warning: Could not open or process image {file_path}. Skipping. Error: {e}")
                continue

        if not frames:
            print("No valid frames could be loaded to create the GIF.")
            return

        # Save as GIF
        # The first frame is the base, and subsequent frames are appended.
        frames[0].save(
            output_gif_path,
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=loop,
            optimize=False # Set to True for smaller file size, but can sometimes introduce artifacts
        )
        print(f"GIF '{output_gif_path}' created successfully from {len(frames)} images.")

    except Exception as e:
        print(f"An error occurred during GIF creation: {e}")

if __name__ == "__main__":
    # Example Usage:
    # 1. Create a dummy folder with some PNG image files for testing
    dummy_image_folder = "temp_png_frames"
    os.makedirs(dummy_image_folder, exist_ok=True)

    # Create some dummy PNG image files
    try:
        from PIL import Image, ImageDraw
        for i in range(15): # Create 15 frames
            img = Image.new('RGB', (200, 200), color = (i*10, 150 - i*5, 200)) # Change color per frame
            d = ImageDraw.Draw(img)
            d.text((50,80), f"Frame {i+1}", fill=(255, 255, 255))
            img.save(os.path.join(dummy_image_folder, f"frame_{i:03d}.png"))
        print(f"Created dummy PNG images in '{dummy_image_folder}'")
    except ImportError:
        print("Pillow not installed or issue creating dummy images. Please manually create some .png images in 'temp_png_frames' for testing.")
        print("Example: frame_001.png, frame_002.png, ...")
        exit() # Exit if PIL is not available to avoid error for dummy image creation


    output_gif_file = "output_animation_pillow.gif"
    frame_duration_ms = 150 # 150 milliseconds per frame (0.15 seconds)

    create_gif_from_pngs_pillow("frames/", output_gif_file, duration=frame_duration_ms)

    # Clean up dummy folder (optional)
    # import shutil
    # shutil.rmtree(dummy_image_folder)
    # print(f"Cleaned up dummy folder: {dummy_image_folder}")


Created dummy PNG images in 'temp_png_frames'
GIF 'output_animation_pillow.gif' created successfully from 130 images.
